In [1]:
from cobra import io
from cobra.core import Metabolite
from cobra import flux_analysis
from Functions import *
from IPython import display
import logging
logging.basicConfig()
logger = logging.getLogger('logger')

#import sbml file
fname = "./../plantcoremetabolism-model/PlantCoreMetabolism_v2_0_0.xml"
model = io.read_sbml_model(fname)
display.clear_output()



#change Biomass_tx to Protein_biomass
rxn = model.reactions.get_by_id("Biomass_tx")
rxn.id = "Protein_biomass_tx"
for met in ["K_b","Ca_b","Mg_b"]:
    met  = model.metabolites.get_by_id(met)
    coeff = rxn.metabolites.get(met)
    rxn.add_metabolites({met:-1*coeff})
met = Metabolite("Protein_b",name="Protein_biomass")
formula_dict = rxn.check_mass_balance()
met.formula = "".join([atom+str(formula_dict[atom]*-1) for atom in formula_dict.keys() if atom != "charge"])
met.charge = formula_dict["charge"]*-1
rxn.add_metabolites({met:1})



model.metabolites.get_by_id("aL_1_PHOSPHATIDYL_GLYCEROL_P_m").remove_from_model()
model.metabolites.get_by_id("aL_1_PHOSPHATIDYL_GLYCEROL_P_p").remove_from_model()

model.reactions.get_by_id("PGPPHOSPHA_RXN_m").add_metabolites({model.metabolites.get_by_id("L_1_PHOSPHATIDYL_GLYCEROL_P_m"):-0.03,
                                                               model.metabolites.get_by_id("PROTON_m"):-0.03})
model.reactions.get_by_id("PHOSPHAGLYPSYN_RXN_m").add_metabolites({model.metabolites.get_by_id("L_1_PHOSPHATIDYL_GLYCEROL_P_m"):0.03,
                                                                   model.metabolites.get_by_id("PROTON_m"):0.03})
model.reactions.get_by_id("PGPPHOSPHA_RXN_p").add_metabolites({model.metabolites.get_by_id("L_1_PHOSPHATIDYL_GLYCEROL_P_p"):-0.03,
                                                               model.metabolites.get_by_id("PROTON_p"):-0.03})
model.reactions.get_by_id("PHOSPHAGLYPSYN_RXN_p").add_metabolites({model.metabolites.get_by_id("L_1_PHOSPHATIDYL_GLYCEROL_P_p"):0.03,
                                                                   model.metabolites.get_by_id("PROTON_p"):0.03})
model.reactions.get_by_id("LPG_biosynthesis_c").add_metabolites({model.metabolites.get_by_id("L_1_PHOSPHATIDYL_GLYCEROL_P_p"):-0.03})

In [2]:
import pandas as pd
biomass = pd.DataFrame(data={"":["sSUCROSE_b","GLC_c","FRU_c","Starch_b","Cellulose_b","Xylan_b",
                                 "L_PHOSPHATIDATE_p","PHOSPHATIDYL_CHOLINE_r",
                                 "L_1_PHOSPHATIDYL_ETHANOLAMINE_r","DIACYLGLYCEROL_p",
                                 "Galactosyl_galactosyl_diacyl_glycerols_p",
                                 "D_Galactosyl_12_diacyl_glycerols_p","2_Lysophosphatidylcholines_r",
                                 "Lysophosphatidylglycerols_r","Triacylglycerols_p",
                                 "L_1_PHOSPHATIDYL_GLYCEROL_p","L_1_phosphatidyl_inositols_r",
                                 "SULFOQUINOVOSYLDIACYLGLYCEROL_p","Protein_b",
                                 "sMAL_b","sCIT_b","sFUM_b","ARG_c","HIS_c","LYS_c","sASP_b",
                                 "sGLU_b","sSER_b","THR_c","ASN_c","sGLN_b","CYS_c",
                                 "GLY_c","PRO_c","sALA_b","VAL_c","ILE_c","LEU_c",
                                 "MET_c","PHE_c","TYR_c","TRP_c","sGABA_b","PALMITATE_p",
                                 "CPD_9245_p","CPD_17412_p","CPD_17291_p","STEARIC_ACID_p",
                                 "OLEATE_CPD_p","Octadecadienoate_p","LINOLENIC_ACID_p",
                                 "ARACHIDIC_ACID_p","CPD_16709_p","DOCOSANOATE_p",
                                 "SUC_c","FUM_c","MAL_c","CIS_ACONITATE_c","CIT_c","MYO_INOSITOL_c",
                                 "pHIS_b","pILE_b","pTHR_b","pARG_b","pASN_b","pGLU_b","pPHE_b",
                                 "pGLN_b","pTYR_b","pMET_b","pASP_b","pVAL_b","pLYS_b","pSER_b",
                                 "pGLY_b","pALA_b","pLEU_b","pPRO_b","pCYS_b","pTRP_b"],
                             "type":[""]*80,
                             "leaf":[0.0]*80,"stem":[0.0]*80,"root":[0.0]*80,"seed":[0.0]*80,},dtype="float64")
biomass = biomass.set_index("")
for i in ["pHIS_b","pILE_b","pTHR_b","pARG_b","pASN_b","pGLU_b","pPHE_b","pGLN_b","pTYR_b","pMET_b",
          "pASP_b","pVAL_b","pLYS_b","pSER_b","pGLY_b","pALA_b","pLEU_b","pPRO_b","pCYS_b","pTRP_b"]:
    biomass.at[i,"type"]="protein"
for i in ["PALMITATE_p","CPD_9245_p","CPD_17412_p","CPD_17291_p","STEARIC_ACID_p",
          "OLEATE_CPD_p","Octadecadienoate_p","LINOLENIC_ACID_p",
          "ARACHIDIC_ACID_p","CPD_16709_p","DOCOSANOATE_p"]:
    biomass.at[i,"type"]="fattyacid"
biomass

,type,leaf,stem,root,seed
,,,,,
sSUCROSE_b,,0.0,0.0,0.0,0.0
GLC_c,,0.0,0.0,0.0,0.0
FRU_c,,0.0,0.0,0.0,0.0
Starch_b,,0.0,0.0,0.0,0.0
Cellulose_b,,0.0,0.0,0.0,0.0
...,...,...,...,...,...
pALA_b,protein,0.0,0.0,0.0,0.0
pLEU_b,protein,0.0,0.0,0.0,0.0
pPRO_b,protein,0.0,0.0,0.0,0.0


In [3]:
proteinMW = model.metabolites.get_by_id("Protein_b").formula_weight

C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 17.2020450391 is not an integer (in formula C17.2020450391H32.8863668319N4.9536245179O9.2182716051S0.1420323181)
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 32.8863668319 is not an integer (in formula C17.2020450391H32.8863668319N4.9536245179O9.2182716051S0.1420323181)
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 4.9536245179 is not an integer (in formula C17.2020450391H32.8863668319N4.9536245179O9.2182716051S0.1420323181)
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 9.2182716051 is not an integer (in formula C17.2020450391H32.8863668319N4.9536245179O9.2182716051S0.1420323181)
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 0.1420323181 is not an integer (in formula C17.2020450391H32.8863668319N4.9536245179O9.2

### Mature leaf

Extract CO2 assimilation rate for PPFD = 600 umol/m2/s between 11:00 and 13:00h from Liu et al 2004, Figure 3

<img src="references/Screenshot_20190717_132513.png" height=50% width=50%>

In [4]:
df_matureleaf_data1 = pd.DataFrame(data={"DAA":[-10.9638294010889,-9.92869328493648,
                                               -9.02599818511797,-7.9920054446461,
                                               -6.98697822141561,-4.05432849364791,
                                               -3.05117967332123,-2.04432849364791,
                                               -1.02035390199637,2.90665154264973,
                                               3.94565335753176,5.86627949183303,
                                               6.86583484573503,9.80232304900181],
                                        "Photosynthesis (umol/m2/s)":[18.4210526315789,18.2395644283121,
                                                                      14.6460980036298,15.989110707804,
                                                                      15.9528130671506,25.7531760435572,
                                                                      28.2214156079855,25.7531760435572,
                                                                      20.4537205081669,24.4464609800363,
                                                                      19.1107078039927,18.2758620689655,
                                                                      25.535390199637,30.2177858439201]})
df_matureleaf_data1

,DAA,Photosynthesis (umol/m2/s)
0,-10.963829,18.421053
1,-9.928693,18.239564
2,-9.025998,14.646098
3,-7.992005,15.989111
4,-6.986978,15.952813
5,-4.054328,25.753176
6,-3.051180,28.221416
7,-2.044328,25.753176
8,-1.020354,20.453721
9,2.906652,24.446461


Extract CO2 assimilation rate for PPFD = 1000 µmol m−2 s−1 from Figure 1 of Weston et al 2011

<img src="references/Screenshot_20190924_144100.png" height=50% width=50%>

In [5]:
df_matureleaf_data2 = pd.DataFrame(data={"Leaf temperature (C)":[24.9632352941176,28.0882352941176,
                                                                 30.0367647058823,32.1691176470588,
                                                                 35.2941176470588,38.2720588235294,
                                                                 40.0367647058824,41.9852941176471],
                                        "Photosynthesis (umol/m2/s)":[18.9105151864667,22.8457003716519,
                                                                      23.5731449442522,25.3471421248238,
                                                                      25.9489939766756,24.2626233499936,
                                                                      21.5252146610278,17.5467768806869]})
df_matureleaf_data2

,Leaf temperature (C),Photosynthesis (umol/m2/s)
0,24.963235,18.910515
1,28.088235,22.845700
2,30.036765,23.573145
3,32.169118,25.347142
4,35.294118,25.948994
5,38.272059,24.262623
6,40.036765,21.525215
7,41.985294,17.546777


In [6]:
df_matureleaf_data3 = pd.DataFrame(data={"Leaf temperature (C)":[20.1102941176471,24.9264705882353,
                                                                 30.1838235294118,35.1102941176471,
                                                                 39.5588235294118],
                                        "Dark respiration (umol/m2/s)":[0.662433155080194,0.965304303539575,
                                                                      1.58024573465748,2.50658899923604,
                                                                      2.75719378660553]})
df_matureleaf_data3

,Leaf temperature (C),Dark respiration (umol/m2/s)
0,20.110294,0.662433
1,24.926471,0.965304
2,30.183824,1.580246
3,35.110294,2.506589
4,39.558824,2.757194


Extract diurnal starch, sucrose, hemicellulose and simple sugar content from Clements 1930, Figure 6, 9, 13 and 14. Based on Figure 1 and 2, dawn = 8am and dusk = 8pm

<img src="references/Screenshot_20190717_161155.png" height=50% width=50%>

<img src="references/Screenshot_20190717_160304.png" height=50% width=50%>

<img src="references/Screenshot_20190717_161259.png" height=50% width=50%>

<img src="references/Screenshot_20190717_160242.png" height=50% width=50%>

In [7]:
df_matureleaf_content_1 = pd.DataFrame(data={"Series":[1,2,3,4],
                                           "starch_dawn (g/100gFW)":[0.4245676464308603,0.4974291547005305,
                                                                     0.8895379007845832,1.2526663271935523],
                                           "hemicellulose_dawn (g/100gFW)":[1.089463383389083,
                                                                            1.8922736237500428,
                                                                            0.8616451473404596,
                                                                            1.1221330617548415],
                                           "simple sugars_dawn (g/100gFW)":[0.2719414988257076,
                                                                            0.6501251398452184,
                                                                            0.5686189976083096,
                                                                            0.5533809766290336],
                                           "sucrose_dawn (g/100gFW)":[0.2719414988257076,0.454127308166961,
                                                                      0.09770538968660913,0],
                                           "starch_dusk (g/100gFW)":[1.3284677247171013,1.600011533626287,
                                                                     2.6254282537133005,1.0978029358571406],
                                           "hemicellulose_dusk (g/100gFW)":[1.089463383389083,
                                                                            0.6633103814170216,
                                                                            0.9510687588805857,
                                                                            1.0721624730031074],
                                           "simple sugars_dusk (g/100gFW)":[0.03202263184115184,
                                                                            0.35337416237039254,
                                                                            0.4138784209550903,
                                                                            0.3891937806184267],
                                           "sucrose_dusk (g/100gFW)":[0.46007757454985465,0.18700390989930638,
                                                                      0.3964371762172485,0.14206905936682546]})
df_matureleaf_content_1.set_index("Series",drop=True,inplace=True)
df_matureleaf_content_1["Δstarch"] = df_matureleaf_content_1["starch_dusk (g/100gFW)"]- \
                                                        df_matureleaf_content_1["starch_dawn (g/100gFW)"]
df_matureleaf_content_1["Δhemicellulose"] = df_matureleaf_content_1["hemicellulose_dusk (g/100gFW)"]- \
                                                        df_matureleaf_content_1["hemicellulose_dawn (g/100gFW)"]
df_matureleaf_content_1["Δsimple sugars"] = df_matureleaf_content_1["simple sugars_dusk (g/100gFW)"]- \
                                                        df_matureleaf_content_1["simple sugars_dawn (g/100gFW)"]
df_matureleaf_content_1["Δsucrose"] = df_matureleaf_content_1["sucrose_dusk (g/100gFW)"]- \
                                                        df_matureleaf_content_1["sucrose_dawn (g/100gFW)"]
df_matureleaf_content_1

,starch_dawn (g/100gFW),hemicellulose_dawn (g/100gFW),simple sugars_dawn (g/100gFW),sucrose_dawn (g/100gFW),starch_dusk (g/100gFW),hemicellulose_dusk (g/100gFW),simple sugars_dusk (g/100gFW),sucrose_dusk (g/100gFW),Δstarch,Δhemicellulose,Δsimple sugars,Δsucrose
Series,,,,,,,,,,,,
1,0.424568,1.089463,0.271941,0.271941,1.328468,1.089463,0.032023,0.460078,0.903900,0.000000,-0.239919,0.188136
2,0.497429,1.892274,0.650125,0.454127,1.600012,0.663310,0.353374,0.187004,1.102582,-1.228963,-0.296751,-0.267123
3,0.889538,0.861645,0.568619,0.097705,2.625428,0.951069,0.413878,0.396437,1.735890,0.089424,-0.154741,0.298732
4,1.252666,1.122133,0.553381,0.000000,1.097803,1.072162,0.389194,0.142069,-0.154863,-0.049971,-0.164187,0.142069


Series 3 ignored because of diurnal fluctuations in metabolite levels were not observed, calculate average from remaining data

In [8]:
df_matureleaf_content_1_std = pd.DataFrame(data={"avg. Δstarch (g/100gFW)":
                                                 [find_average(df_matureleaf_content_1["Δstarch"])],
                                                 "avg. Δstarch (%)":
                                                 [100*find_average(df_matureleaf_content_1["Δstarch"]/ \
                                                               df_matureleaf_content_1["starch_dusk (g/100gFW)"])],
                                                 "avg. Δsimple sugars (g/100gFW)":
                                                 [find_average(df_matureleaf_content_1["Δsimple sugars"])],
                                                 "avg. Δsucrose (g/100gFW)":
                                                 [find_average(df_matureleaf_content_1["Δsucrose"])],
                                                 "avg. Δsucrose (%)":
                                                  [100*find_average(df_matureleaf_content_1["Δsucrose"]/ \
                                                                df_matureleaf_content_1["sucrose_dusk (g/100gFW)"])]})
df_matureleaf_content_1_std

,avg. Δstarch (g/100gFW),avg. Δstarch (%),avg. Δsimple sugars (g/100gFW),avg. Δsucrose (g/100gFW),avg. Δsucrose (%)
0,0.896877,47.240852,-0.213899,0.090453,18.350658


In [9]:
print("Starch to sucrose ratio = "+str(float(df_matureleaf_content_1_std["avg. Δstarch (g/100gFW)"])/ \
                                             model.metabolites.GLC_c.formula_weight/ \
                                      (float(df_matureleaf_content_1_std["avg. Δsucrose (g/100gFW)"])/ \
                                             model.metabolites.SUCROSE_c.formula_weight)))

Starch to sucrose ratio = 18.839190878931536


### Leaf

Extract sucrose,hexose and starch content from Liu et al 2004, Figure 4 (PPFD = 600 umol/m2/s, photoperiod=12h, sampled between 11:00 and 13:00 h into photoperiod)

<img src="references/Screenshot_20190717_135552.png" height=50% width=50%>

In [10]:
df_leaf_content_1 = pd.DataFrame(data={"DAA":[0,3,5,7,10],
                                       "sucrose (mg/gDW)":[6.3492063492063275,5.0793650793650515,
                                                           5.0793650793650515,4.20634920634918,
                                                           1.746031746031723],
                                       "hexose (mg/gDW)":[61.64383561643821,81.36986301369862,
                                                          82.46575342465738,69.3150684931507,
                                                          45.5707762557078],
                                       "starch (mg/gDW)":[47.42268041237085,47.01030927835024,
                                                          37.525773195875956,35.25773195876279,
                                                          64.32989690721634]})
df_leaf_content_1.set_index("DAA",inplace=True,drop=True)
df_leaf_content_1

,sucrose (mg/gDW),hexose (mg/gDW),starch (mg/gDW)
DAA,,,
0,6.349206,61.643836,47.422680
3,5.079365,81.369863,47.010309
5,5.079365,82.465753,37.525773
7,4.206349,69.315068,35.257732
10,1.746032,45.570776,64.329897


Using Clements 1930 data on starch accumulated in 12h to estimate starch, hexose and sucrose content when diel fluctuations are ignored.

In [11]:
df_leaf_content_1_revised = pd.DataFrame(data={"sucrose (mg/gDW)":df_leaf_content_1["sucrose (mg/gDW)"]* \
                                                  (100-df_matureleaf_content_1_std["avg. Δsucrose (%)"][0])/100,
                                           "starch (mg/gDW)":df_leaf_content_1["starch (mg/gDW)"]* \
                                                  (100-df_matureleaf_content_1_std["avg. Δstarch (%)"][0])/100,
                                           "hexose (mg/gDW)":df_leaf_content_1["hexose (mg/gDW)"]})
df_leaf_content_1_revised

,sucrose (mg/gDW),starch (mg/gDW),hexose (mg/gDW)
DAA,,,
0,5.184085,25.019802,61.643836
3,4.147268,24.802239,81.369863
5,4.147268,19.798278,82.465753
7,3.434456,18.601679,69.315068
10,1.425623,33.939906,45.570776


Gather stach, hexose and sucrose content, 8hrs after illumination from Huber 2008, Table 1

In [12]:
df_leaf_content_2 = pd.DataFrame(data={"cultivar":["Maple Presto","Ransom"],
                                       "starch (mg/dm2)":[190,78],
                                       "hexose (mg/dm2)":[0,1.9],
                                       "sucrose (mg/dm2)":[8.7,4.9]})
df_leaf_content_2.set_index("cultivar",drop=True,inplace=True)
df_leaf_content_2

,starch (mg/dm2),hexose (mg/dm2),sucrose (mg/dm2)
cultivar,,,
Maple Presto,190,0.0,8.7
Ransom,78,1.9,4.9


Extract amino acid content from Chiozza et al 2010, Figure 2

<img src="references/Screenshot_20190717_143619.png" height=50% width=50%>

In [13]:
df_leaf_content_3 = pd.DataFrame(data={"Variety":["aphid resitant","aphid sensitive"],
                                       "GABA (nmol/50mgFW)":[0.8287671232876719,1.0136986301369868],
                                       "Asn (nmol/50mgFW)":[67.26027397260277,86.1643835616437],
                                       "Gln (nmol/50mgFW)":[82.87671232876716,99.31506849315056],
                                       "Glt (nmol/50mgFW)":[184.58904109589022,234.93150684931462],
                                       "His (nmol/50mgFW)":[11.778523489932848,13.657718120805297],
                                       "Pro (nmol/50mgFW)":[19.062499999999943,20.68181818181808],
                                       "Ser (nmol/50mgFW)":[87.28205128205093,100.4102564102559]})
df_leaf_content_3.set_index("Variety",drop=True,inplace=True)
df_leaf_content_3

,GABA (nmol/50mgFW),Asn (nmol/50mgFW),Gln (nmol/50mgFW),Glt (nmol/50mgFW),His (nmol/50mgFW),Pro (nmol/50mgFW),Ser (nmol/50mgFW)
Variety,,,,,,,
aphid resitant,0.828767,67.260274,82.876712,184.589041,11.778523,19.062500,87.282051
aphid sensitive,1.013699,86.164384,99.315068,234.931507,13.657718,20.681818,100.410256


Use Clements 1930 data to extract baseline sucrose, simple sugars, starch and hemicellulose data from diel metabolite levels

In [14]:
xlist = list()
ylist = list()
ylist2 = list()
ylist3 = list()
ylist4 = list()
for i in df_matureleaf_content_1.index:
    if i == 4:
        continue
    xlist.append(i)
    ylist.append(min(df_matureleaf_content_1["simple sugars_dawn (g/100gFW)"][i],
                     df_matureleaf_content_1["simple sugars_dusk (g/100gFW)"][i]))
    ylist2.append(min(df_matureleaf_content_1["sucrose_dawn (g/100gFW)"][i],
                      df_matureleaf_content_1["sucrose_dusk (g/100gFW)"][i]))
    ylist3.append(min(df_matureleaf_content_1["hemicellulose_dawn (g/100gFW)"][i],
                      df_matureleaf_content_1["hemicellulose_dusk (g/100gFW)"][i]))
    ylist4.append(min(df_matureleaf_content_1["starch_dawn (g/100gFW)"][i],
                      df_matureleaf_content_1["starch_dusk (g/100gFW)"][i]))
df_leaf_content_4 = pd.DataFrame(data={"Series":xlist,
                                       "baseline starch (g/100gFW)":ylist4,
                                       "baseline hemicellulose (g/100gFW)":ylist3,
                                       "baseline sucrose (g/100gFW)":ylist2,
                                       "baseline simple sugars (g/100gFW)":ylist})
df_leaf_content_4.set_index("Series",drop=True,inplace=True)
df_leaf_content_4

,baseline starch (g/100gFW),baseline hemicellulose (g/100gFW),baseline sucrose (g/100gFW),baseline simple sugars (g/100gFW)
Series,,,,
1,0.424568,1.089463,0.271941,0.032023
2,0.497429,0.663310,0.187004,0.353374
3,0.889538,0.861645,0.097705,0.413878


##### Data for soy sample analysis. Thanks to Yu and Pedro   #####
Starch data found here: Onedrive > FFAR > Soy and wheat biomass content > Soy > Starch.xlsx 
  
*x* nmol/mL/gDW = *100x* nmol/gDW since 100mL was collected from 1gDW of sample. 

In [15]:

df_content_5_starch = pd.DataFrame(data={"sample":["leaves","stems","roots","leaves elevated CO2","stems elevated CO2","roots elevated CO2"],
                                         "Glucose (ng/mL/mgDW)":[6453.33263,4937.627288,3292.909588,6736.354398,6747.968055,4805.3406]})
df_content_5_starch.set_index("sample",drop=True,inplace=True)
df_content_5_starch_proc = pd.DataFrame()
df_content_5_starch_proc["Glucose (ug/gDW)"]=0.1*df_content_5_starch["Glucose (ng/mL/mgDW)"]

df_content_5_starch_proc

,Glucose (ug/gDW)
sample,
leaves,645.333263
stems,493.762729
roots,329.290959
leaves elevated CO2,673.635440
stems elevated CO2,674.796806
roots elevated CO2,480.534060


Cell wall data found here: Onedrive > FFAR > Soy and wheat biomass content > Soy > Cell_wall.xlsx  

  
*x* nmol/mL/gDW = *100x* nmol/gDW since 100mL was collected from 1gDW of sample. 

In [16]:

df_content_5_cellwall = pd.DataFrame(data={"sample":["leaves","stems","roots","leaves elevated CO2","stems elevated CO2","roots elevated CO2"],
                                                "Mannose (ng/mL/mgDW)":[909.347203,1193.375478,1167.489003,788.4246863,1115.565697,1199.198745],
                                                "Arabinose (ng/mL/mgDW)":[4177.779617,3980.129857,3783.536424,3600.710558,4174.145023,3847.448145],
                                                "Rhamnose (ng/mL/mgDW)":[486.5496815,791.8369,827.138705,442.9417995,731.397256,789.5078091],
                                                "Xylose (ng/mL/mgDW)":[4435.45749,16543.75469,17195.08294,3868.961062,16104.30023,18766.39696],
                                                "Galactose (ng/mL/mgDW)":[1774.797419,1164.283349,813.3414433,1944.430924,1803.057221,1043.642511],
                                                "Glucose (ng/mL/mgDW)":[619.3173969,115.9412739,85.60903026,861.9915986,233.3273322,158.5507089],
                                                "Fucose (ng/mL/mgDW)":[359.4060706,246.3683858,207.9966481,423.6423199,561.4443752,377.3402891]})

df_content_5_cellwall.set_index("sample",drop=True,inplace=True)
df_content_5_cellwall_proc = pd.DataFrame()
temp = list(df_content_5_cellwall.columns)
for col in temp:
    df_content_5_cellwall_proc[col.replace("(ng/mL/mgDW)","(ug/gDW)")]=0.1*df_content_5_cellwall[col]

df_content_5_cellwall_proc

,Mannose (ug/gDW),Arabinose (ug/gDW),Rhamnose (ug/gDW),Xylose (ug/gDW),Galactose (ug/gDW),Glucose (ug/gDW),Fucose (ug/gDW)
sample,,,,,,,
leaves,90.934720,417.777962,48.654968,443.545749,177.479742,61.931740,35.940607
stems,119.337548,398.012986,79.183690,1654.375469,116.428335,11.594127,24.636839
roots,116.748900,378.353642,82.713870,1719.508294,81.334144,8.560903,20.799665
leaves elevated CO2,78.842469,360.071056,44.294180,386.896106,194.443092,86.199160,42.364232
stems elevated CO2,111.556570,417.414502,73.139726,1610.430023,180.305722,23.332733,56.144438
roots elevated CO2,119.919874,384.744815,78.950781,1876.639696,104.364251,15.855071,37.734029



soluble metabolites data found here: Onedrive > FFAR > Soy and wheat biomass content > Soy > Sol_Metab.xlsx  

  
*x* nmol/mL/gDW = *100x* nmol/gDW since 100mL was collected from 1gDW of sample. 

In [17]:
df_content_5_soluble = pd.DataFrame(data={"sample":["leaves","stems","roots","leaves elevated CO2","stems elevated CO2","roots elevated CO2"],
                                          "Alanine (ng/mL/mgDW)":[0.2649,0.1163,0.1781,0.2769,0.1108,0.1376],
                                          "Valine (ng/mL/mgDW)":[0.0298,0.0115,0.0205,0.0281,0.0116,0.0096],
                                          "Leucine (ng/mL/mgDW)":[0.1398,0.0278,0.0374,0.0888,0.0206,0.0192],
                                          "Isoleucine (ng/mL/mgDW)":[0.0254,0.0112,0.0148,0.0238,0.0068,0.0071],
                                          "Proline (ng/mL/mgDW)":[0.0508,0.0079,0.0161,0.0475,0.0087,0.0064],
                                          "Glycine (ng/mL/mgDW)":[0.0161,0.0138,0.0065,0.0176,0.0115,0.0122],
                                          "Succinate (ng/mL/mgDW)":[0.0123,0.0139,0.0097,0.0136,0.0120,0.0101],
                                          "Fumarate (ng/mL/mgDW)":[0.2997,0.2642,0.1211,0.2587,0.1885,0.0716],
                                          "Serine (ng/mL/mgDW)":[0.0996,0.0691,0.0743,0.1088,0.0615,0.0505],
                                          "Threonine (ng/mL/mgDW)":[0.0853,0.0889,0.0676,0.1091,0.0875,0.0518],
                                          "Methionine (ng/mL/mgDW)":[0.0466,0.0119,0.0072,0.0366,0.0148,0.0116],
                                          "Malate (ng/mL/mgDW)":[0.2157,0.1077,0.2220,0.2250,0.0826,0.1552],
                                          "Aspartate (ng/mL/mgDW)":[0.2129,0.0776,0.0844,0.1981,0.0664,0.0606],
                                          "pyroGlutamate (ng/mL/mgDW)":[0.1582,0.0516,0.0682,0.1479,0.0529,0.0444],
                                          "GABA (ng/mL/mgDW)":[0.1218,0.0610,0.0750,0.1092,0.0411,0.0490],
                                          "Cysteine (ng/mL/mgDW)":[0.1975,0.0928,0.0709,0.2291,0.0772,0.0514],
                                          "Phenylalanine (ng/mL/mgDW)":[0.0305,0.0098,0.0092,0.0260,0.0060,0.0070],
                                          "Glutamate (ng/mL/mgDW)":[0.3218,0.1226,0.1692,0.3189,0.1046,0.1058],
                                          "Ribose (ng/mL/mgDW)":[0.3618,0.1912,0.3199,0.2461,0.1421,0.2336],
                                          "Lyxose (ng/mL/mgDW)":[0.0486,0.0607,0.0428,0.0555,0.0372,0.0355],
                                          "Rhamnose (ng/mL/mgDW)":[0.0396,0.0166,0.0137,0.0280,0.0121,0.0097],
                                          "Putrescine (ng/mL/mgDW)":[0.0309,0.0329,0.0923,0.0526,0.0284,0.0512],
                                          "Citrate (ng/mL/mgDW)":[0.1658,0.0344,0.0682,0.1783,0.0395,0.0574],
                                          "Fructose (ng/mL/mgDW)":[1.0838,1.5821,1.7199,1.3719,0.7860,1.1154],
                                          "Galactose (ng/mL/mgDW)":[0.1752,1.3062,0.2582,0.2102,1.3605,0.1663],
                                          "Glucose (ng/mL/mgDW)":[0.0624,0.1929,0.0922,0.0744,0.4834,0.0594],
                                          "Lysine (ng/mL/mgDW)":[0.0033,0.0144,0.0138,0.0096,0.0084,0.0036],
                                          "Tyrosine (ng/mL/mgDW)":[0.1134,0.0818,0.0541,0.1144,0.0668,0.0464],
                                          "myoInositol (ng/mL/mgDW)":[0.1267,0.0461,0.0750,0.1247,0.0351,0.0561],
                                          "Sucrose (ng/mL/mgDW)":[0.0228,0.0242,0.0213,0.0190,0.0233,0.0179]})
df_content_5_soluble.set_index("sample",drop=True,inplace=True)
df_content_5_soluble_proc = pd.DataFrame()
temp = list(df_content_5_soluble.columns)
for col in temp:
    df_content_5_soluble_proc[col.replace("(ng/mL/mgDW)","(ug/gDW)")]=0.1*df_content_5_soluble[col]

df_content_5_soluble_proc

,Alanine (ug/gDW),Valine (ug/gDW),Leucine (ug/gDW),Isoleucine (ug/gDW),Proline (ug/gDW),Glycine (ug/gDW),Succinate (ug/gDW),Fumarate (ug/gDW),Serine (ug/gDW),Threonine (ug/gDW),...,Rhamnose (ug/gDW),Putrescine (ug/gDW),Citrate (ug/gDW),Fructose (ug/gDW),Galactose (ug/gDW),Glucose (ug/gDW),Lysine (ug/gDW),Tyrosine (ug/gDW),myoInositol (ug/gDW),Sucrose (ug/gDW)
sample,,,,,,,,,,,,,,,,,,,,,
leaves,0.02649,0.00298,0.01398,0.00254,0.00508,0.00161,0.00123,0.02997,0.00996,0.00853,...,0.00396,0.00309,0.01658,0.10838,0.01752,0.00624,0.00033,0.01134,0.01267,0.00228
stems,0.01163,0.00115,0.00278,0.00112,0.00079,0.00138,0.00139,0.02642,0.00691,0.00889,...,0.00166,0.00329,0.00344,0.15821,0.13062,0.01929,0.00144,0.00818,0.00461,0.00242
roots,0.01781,0.00205,0.00374,0.00148,0.00161,0.00065,0.00097,0.01211,0.00743,0.00676,...,0.00137,0.00923,0.00682,0.17199,0.02582,0.00922,0.00138,0.00541,0.00750,0.00213
leaves elevated CO2,0.02769,0.00281,0.00888,0.00238,0.00475,0.00176,0.00136,0.02587,0.01088,0.01091,...,0.00280,0.00526,0.01783,0.13719,0.02102,0.00744,0.00096,0.01144,0.01247,0.00190
stems elevated CO2,0.01108,0.00116,0.00206,0.00068,0.00087,0.00115,0.00120,0.01885,0.00615,0.00875,...,0.00121,0.00284,0.00395,0.07860,0.13605,0.04834,0.00084,0.00668,0.00351,0.00233
roots elevated CO2,0.01376,0.00096,0.00192,0.00071,0.00064,0.00122,0.00101,0.00716,0.00505,0.00518,...,0.00097,0.00512,0.00574,0.11154,0.01663,0.00594,0.00036,0.00464,0.00561,0.00179



Fatty acid data found here: Onedrive > FFAR > Soy and wheat biomass content > Soy > Soybean_fattyAcids_analysis.xlsx 

  
*x* nmol/mL/gDW = *100x* nmol/gDW since 100mL was collected from 1gDW of sample. 

In [18]:
df_content_5_FA = pd.DataFrame(data={"sample":["leaves","stems","roots","leaves elevated CO2","stems elevated CO2","roots elevated CO2"],
                                          "C8:0 (ng/mL/mgDW)":[74.02,1.28,25.71,62.24,3.66,4.39],
                                          "C10:0 (ng/mL/mgDW)":[1.31,0.03,0.03,0.25,-0.04,0.1],
                                          "C12:0 (ng/mL/mgDW)":[45.08,0.44,0.03,76.96,0.92,0.06],
                                          "C13:0 (ng/mL/mgDW)":[1.53,0.79,0.89,0.98,1.65,1.49],
                                          "C14:1z-9 (ng/mL/mgDW)":[18.76,2.3,11.36,29.45,4.45,10.88],
                                          "C14:0 (ng/mL/mgDW)":[301.48,53.42,25.9,358.06,62.32,34.74],
                                          "C15:0 (ng/mL/mgDW)":[16.28,86.9,86.73,59.72,63.44,41.87],
                                          "C16:1z-9 (ng/mL/mgDW)":[305.25,4.5,228.71,467.55,-10.78,66.19],
                                          "C16:0 (ng/mL/mgDW)":[11540.51,4979.04,4970.88,11577.99,6215.14,4179.99],
                                          "C17:0 (ng/mL/mgDW)":[617.66,695.29,785.86,909.9,810.46,389.29],
                                          "C18:2z-9z-12 (ng/mL/mgDW)":[19650.02,3859.86,3988.7,20144.98,4178.83,3572.52],
                                          "C18:1z-9 (ng/mL/mgDW)":[30947.65,3065.81,1371.35,31562.05,3154.44,2782.39],
                                          "C18:1e-9 (ng/mL/mgDW)":[3774.26,1040.73,1743.93,4557.51,1115.53,2344.13],
                                          "C18:0 (ng/mL/mgDW)":[22260.01,8871.85,6755.09,25165.16,11760,4936.13],
                                          "C20:1z-13 (ng/mL/mgDW)":[552.27,152.14,146.37,631.21,253.24,240.66],
                                          "C20:0 (ng/mL/mgDW)":[604.81,363.55,460.63,742.01,402.5,509.2],
                                          "C22:1z-13 (ng/mL/mgDW)":[5.4,4.72,8.32,15.67,6.09,11.97],
                                          "C22:0 (ng/mL/mgDW)":[527.94,335.08,1218.94,636.57,361.63,1301.01]})

df_content_5_FA.set_index("sample",drop=True,inplace=True)
df_content_5_FA_proc = pd.DataFrame()
temp = list(df_content_5_FA.columns)
for col in temp:
    df_content_5_FA_proc[col.replace("(ng/mL/mgDW)","(ug/gDW)")]=0.1*df_content_5_FA[col]

df_content_5_FA_proc

,C8:0 (ug/gDW),C10:0 (ug/gDW),C12:0 (ug/gDW),C13:0 (ug/gDW),C14:1z-9 (ug/gDW),C14:0 (ug/gDW),C15:0 (ug/gDW),C16:1z-9 (ug/gDW),C16:0 (ug/gDW),C17:0 (ug/gDW),C18:2z-9z-12 (ug/gDW),C18:1z-9 (ug/gDW),C18:1e-9 (ug/gDW),C18:0 (ug/gDW),C20:1z-13 (ug/gDW),C20:0 (ug/gDW),C22:1z-13 (ug/gDW),C22:0 (ug/gDW)
sample,,,,,,,,,,,,,,,,,,
leaves,7.402,0.131,4.508,0.153,1.876,30.148,1.628,30.525,1154.051,61.766,1965.002,3094.765,377.426,2226.001,55.227,60.481,0.540,52.794
stems,0.128,0.003,0.044,0.079,0.230,5.342,8.690,0.450,497.904,69.529,385.986,306.581,104.073,887.185,15.214,36.355,0.472,33.508
roots,2.571,0.003,0.003,0.089,1.136,2.590,8.673,22.871,497.088,78.586,398.870,137.135,174.393,675.509,14.637,46.063,0.832,121.894
leaves elevated CO2,6.224,0.025,7.696,0.098,2.945,35.806,5.972,46.755,1157.799,90.990,2014.498,3156.205,455.751,2516.516,63.121,74.201,1.567,63.657
stems elevated CO2,0.366,-0.004,0.092,0.165,0.445,6.232,6.344,-1.078,621.514,81.046,417.883,315.444,111.553,1176.000,25.324,40.250,0.609,36.163
roots elevated CO2,0.439,0.010,0.006,0.149,1.088,3.474,4.187,6.619,417.999,38.929,357.252,278.239,234.413,493.613,24.066,50.920,1.197,130.101



Lipid content data found here: Onedrive > FFAR > Soy and wheat biomass content > Soy > Soybean_fattyAcids_analysis.xlsx 

  
*x* nmol/mL/gDW = *100x* nmol/gDW since 100mL was collected from 1gDW of sample. 

In [19]:
df_content_5_PL = pd.DataFrame(data={"sample":["leaves","stems","roots","leaves elevated CO2",
                                               "stems elevated CO2","roots elevated CO2"],
                                     "Lipid (ng/mL/mgDW)":[0.087378971,0.037781009,0.023859039,
                                                           0.083184314,0.013976472,0.036703397]})

df_content_5_PL.set_index("sample",drop=True,inplace=True)
df_content_5_PL_proc = pd.DataFrame()
temp = list(df_content_5_PL.columns)
for col in temp:
    df_content_5_PL_proc[col.replace("(ng/mL/mgDW)","(ug/gDW)")]=0.1*df_content_5_PL[col]

df_content_5_PL_proc

,Lipid (ug/gDW)
sample,
leaves,0.008738
stems,0.003778
roots,0.002386
leaves elevated CO2,0.008318
stems elevated CO2,0.001398
roots elevated CO2,0.003670


In [20]:
df_content_5_ProteinAA = pd.DataFrame(data={"sample":["leaves","stems","roots","leaves elevated CO2",
                                               "stems elevated CO2","roots elevated CO2"],
                                            "L-Alanine (ng/mL/mgDW)":[3562.724563,3604.639967,2707.892756,
                                                                      2603.930552,2628.072344,2473.827426],
                                            "Glycine (ng/mL/mgDW)":[2936.816023,3140.591031,2244.478424,
                                                                    2306.387509,2193.417556,1981.980494],
                                            "L-Valine (ng/mL/mgDW)":[3300.758848,3209.19445,2708.38747,
                                                                     2301.578047,2533.4362,2356.34864],
                                            "L-Leucine (ng/mL/mgDW)":[4228.775408,3796.048019,2968.023929,
                                                                      3258.833035,2724.219009,2567.989851],
                                            "Isoleucine (ng/mL/mgDW)":[2427.986014,2716.430811,2206.273758,
                                                                       1773.965673,2056.324308,1869.131462],
                                            "L-Proline (ng/mL/mgDW)":[2940.325455,3502.706159,2640.555183,
                                                                      2055.501205,2611.536476,2304.625942],
                                            "L-Methionine (ng/mL/mgDW)":[2682.686757,800.1521879,1016.992457,
                                                                         1945.718777,1326.384186,1188.32779],
                                            "L-Serine (ng/mL/mgDW)":[3246.306664,2862.542014,2296.621494,
                                                                     2311.306026,2134.755587,2030.703317],
                                            "L-Threonine (ng/mL/mgDW)":[4792.560457,2260.917193,2218.225283,
                                                                        3177.587023,2339.924653,2245.522082],
                                            "L-Phenylalanine (ng/mL/mgDW)":[3862.179976,2731.192372,2468.096919,
                                                                            2718.459498,2252.055638,2210.645096],
                                            "L-Aspartic acid (ng/mL/mgDW)":[4864.620389,4111.25701,3010.306205,
                                                                            3511.880167,3195.199592,2517.374484],
                                            "L-Cysteine (ng/mL/mgDW)":[466.5008967,178.8914881,142.4011592,
                                                                       408.8298836,210.0511611,166.7736588],
                                            "L-Glutamic acid (ng/mL/mgDW)":[5434.708562,3230.31375,2588.437246,
                                                                            3994.430081,2681.077972,2561.870237],
                                            "L-Asparagine (ng/mL/mgDW)":[252.1503451,6.55943952,0.09113922,
                                                                         49.2544078,4.73348119,09.6220968],
                                            "L-Lysine (ng/mL/mgDW)":[35253.38836,0856.94202,1026.32059,
                                                                     6457.52025,3589.38524,1972.21797],
                                            "L-Glutamine (ng/mL/mgDW)":[63458.12061,252.996115,325.502846,
                                                                        0097.7828,627.819528,894.078784],
                                            "L-Tyrosine (ng/mL/mgDW)":[5139.620495,059.115994,813.058894,
                                                                       526.922917,339.855083,967.739371],
                                            "L-Tryptophan (ng/mL/mgDW)":[63.64638109,5.25723804,4.02637037,
                                                                         6.99212082,5.82512027,4.69409128]})

df_content_5_ProteinAA.set_index("sample",drop=True,inplace=True)
df_content_5_ProteinAA_proc = pd.DataFrame()
temp = list(df_content_5_ProteinAA.columns)
for col in temp:
    df_content_5_ProteinAA_proc[col.replace("(ng/mL/mgDW)","(ug/gDW)")]=0.1*df_content_5_ProteinAA[col]

df_content_5_ProteinAA_proc

,L-Alanine (ug/gDW),Glycine (ug/gDW),L-Valine (ug/gDW),L-Leucine (ug/gDW),Isoleucine (ug/gDW),L-Proline (ug/gDW),L-Methionine (ug/gDW),L-Serine (ug/gDW),L-Threonine (ug/gDW),L-Phenylalanine (ug/gDW),L-Aspartic acid (ug/gDW),L-Cysteine (ug/gDW),L-Glutamic acid (ug/gDW),L-Asparagine (ug/gDW),L-Lysine (ug/gDW),L-Glutamine (ug/gDW),L-Tyrosine (ug/gDW),L-Tryptophan (ug/gDW)
sample,,,,,,,,,,,,,,,,,,
leaves,356.272456,293.681602,330.075885,422.877541,242.798601,294.032546,268.268676,324.630666,479.256046,386.217998,486.462039,46.650090,543.470856,25.215035,3525.338836,6345.812061,513.962050,6.364638
stems,360.463997,314.059103,320.919445,379.604802,271.643081,350.270616,80.015219,286.254201,226.091719,273.119237,411.125701,17.889149,323.031375,0.655944,85.694202,25.299612,5.911599,0.525724
roots,270.789276,224.447842,270.838747,296.802393,220.627376,264.055518,101.699246,229.662149,221.822528,246.809692,301.030620,14.240116,258.843725,0.009114,102.632059,32.550285,81.305889,0.402637
leaves elevated CO2,260.393055,230.638751,230.157805,325.883304,177.396567,205.550121,194.571878,231.130603,317.758702,271.845950,351.188017,40.882988,399.443008,4.925441,645.752025,9.778280,52.692292,0.699212
stems elevated CO2,262.807234,219.341756,253.343620,272.421901,205.632431,261.153648,132.638419,213.475559,233.992465,225.205564,319.519959,21.005116,268.107797,0.473348,358.938524,62.781953,33.985508,0.582512
roots elevated CO2,247.382743,198.198049,235.634864,256.798985,186.913146,230.462594,118.832779,203.070332,224.552208,221.064510,251.737448,16.677366,256.187024,0.962210,197.221797,89.407878,96.773937,0.469409


### Summary

- Since amino acid content was only available in g/gFW and FW/DW data was not available, diel starch and sucrose levels (in g/gFW) were used to predict baseline starch and sucrose levels in leaf


In [21]:
# biomass.at["sSUCROSE_b","leaf"]=df_content_5_soluble_proc["Sucrose (ug/gDW)"]["leaves"]/ \
#                                 model.metabolites.get_by_id("SUCROSE_c").formula_weight
# biomass.at["GLC_c","leaf"]=df_content_5_soluble_proc["Glucose (ug/gDW)"]["leaves"]/ \
#                            model.metabolites.get_by_id("GLC_c").formula_weight
# biomass.at["FRU_c","leaf"]=df_content_5_soluble_proc["Fructose (ug/gDW)"]["leaves"]/ \
#                            model.metabolites.get_by_id("FRU_c").formula_weight

# biomass.at["Starch_b","leaf"]=df_content_5_starch_proc["Glucose (ug/gDW)"]["leaves"]/ \
#                               model.metabolites.get_by_id("GLC_c").formula_weight

# biomass.at["Cellulose_b","leaf"]=df_content_5_cellwall_proc["Mannose (ug/gDW)"]["leaves"]/ \
#                                  model.metabolites.get_by_id("MANNOSE_c").formula_weight
# biomass.at["Xylan_b","leaf"]=df_content_5_cellwall_proc["Xylose (ug/gDW)"]["leaves"]/ \
#                              model.metabolites.get_by_id("Xylan_b").formula_weight

# biomass.at["L_PHOSPHATIDATE_p","leaf"]=df_content_5_PL_proc["Lipid (ug/gDW)"]["leaves"]/ \
#                                        model.metabolites.get_by_id("L_PHOSPHATIDATE_p").formula_weight
    
# biomass.at["sMAL_b","leaf"]=df_content_5_soluble_proc["Malate (ug/gDW)"]["leaves"]/ \
#                             model.metabolites.get_by_id("MAL_c").formula_weight
# biomass.at["sCIT_b","leaf"]=df_content_5_soluble_proc["Citrate (ug/gDW)"]["leaves"]/ \
#                             model.metabolites.get_by_id("CIT_c").formula_weight
# biomass.at["sFUM_b","leaf"]=df_content_5_soluble_proc["Fumarate (ug/gDW)"]["leaves"]/ \
#                             model.metabolites.get_by_id("FUM_c").formula_weight

# biomass.at["VAL_c","leaf"]=df_content_5_soluble_proc["Valine (ug/gDW)"]["leaves"]/ \
#                            model.metabolites.get_by_id("VAL_c").formula_weight
# biomass.at["sALA_b","leaf"]=df_content_5_soluble_proc["Alanine (ug/gDW)"]["leaves"]/ \
#                             model.metabolites.get_by_id("sALA_b").formula_weight
# biomass.at["LEU_c","leaf"]=df_content_5_soluble_proc["Leucine (ug/gDW)"]["leaves"]/ \
#                            model.metabolites.get_by_id("LEU_c").formula_weight
# biomass.at["ILE_c","leaf"]=df_content_5_soluble_proc["Isoleucine (ug/gDW)"]["leaves"]/ \
#                            model.metabolites.get_by_id("ILE_c").formula_weight
# biomass.at["PRO_c","leaf"]=df_content_5_soluble_proc["Proline (ug/gDW)"]["leaves"]/ \
#                            model.metabolites.get_by_id("PRO_c").formula_weight
# biomass.at["GLY_c","leaf"]=df_content_5_soluble_proc["Glycine (ug/gDW)"]["leaves"]/ \
#                            model.metabolites.get_by_id("GLY_c").formula_weight
# biomass.at["SUC_c","leaf"]=df_content_5_soluble_proc["Succinate (ug/gDW)"]["leaves"]/ \
#                             model.metabolites.get_by_id("SUC_c").formula_weight
# biomass.at["sSER_b","leaf"]=df_content_5_soluble_proc["Serine (ug/gDW)"]["leaves"]/ \
#                             model.metabolites.get_by_id("sSER_b").formula_weight
# biomass.at["THR_c","leaf"]=df_content_5_soluble_proc["Threonine (ug/gDW)"]["leaves"]/ \
#                            model.metabolites.get_by_id("THR_c").formula_weight
# biomass.at["MET_c","leaf"]=df_content_5_soluble_proc["Methionine (ug/gDW)"]["leaves"]/ \
#                            model.metabolites.get_by_id("MET_c").formula_weight
# biomass.at["sASP_b","leaf"]=df_content_5_soluble_proc["Aspartate (ug/gDW)"]["leaves"]/ \
#                             model.metabolites.get_by_id("sASP_b").formula_weight
# biomass.at["sGABA_b","leaf"]=df_content_5_soluble_proc["GABA (ug/gDW)"]["leaves"]/ \
#                              model.metabolites.get_by_id("sGABA_b").formula_weight
# biomass.at["CYS_c","leaf"]=df_content_5_soluble_proc["Cysteine (ug/gDW)"]["leaves"]/ \
#                            model.metabolites.get_by_id("CYS_c").formula_weight
# biomass.at["PHE_c","leaf"]=df_content_5_soluble_proc["Phenylalanine (ug/gDW)"]["leaves"]/ \
#                            model.metabolites.get_by_id("PHE_c").formula_weight
# biomass.at["sGLU_b","leaf"]=df_content_5_soluble_proc["Glutamate (ug/gDW)"]["leaves"]/ \
#                             model.metabolites.get_by_id("sGLU_b").formula_weight
# biomass.at["sCIT_b","leaf"]=df_content_5_soluble_proc["Citrate (ug/gDW)"]["leaves"]/ \
#                             model.metabolites.get_by_id("sCIT_b").formula_weight
# biomass.at["LYS_c","leaf"]=df_content_5_soluble_proc["Lysine (ug/gDW)"]["leaves"]/ \
#                            model.metabolites.get_by_id("LYS_c").formula_weight
# biomass.at["TYR_c","leaf"]=df_content_5_soluble_proc["Tyrosine (ug/gDW)"]["leaves"]/ \
#                            model.metabolites.get_by_id("TYR_c").formula_weight
# biomass.at["MYO_INOSITOL_c","leaf"]=df_content_5_soluble_proc["myoInositol (ug/gDW)"]["leaves"]/ \
#                                     model.metabolites.get_by_id("MYO_INOSITOL_c").formula_weight


# biomass.at["pALA_b","leaf"]=df_content_5_ProteinAA_proc["L-Alanine (ug/gDW)"]["leaves"]/ \
#                             model.metabolites.get_by_id("L_ALPHA_ALANINE_c").formula_weight
# biomass.at["pGLY_b","leaf"]=df_content_5_ProteinAA_proc["Glycine (ug/gDW)"]["leaves"]/ \
#                             model.metabolites.get_by_id("GLY_c").formula_weight
# biomass.at["pVAL_b","leaf"]=df_content_5_ProteinAA_proc["L-Valine (ug/gDW)"]["leaves"]/ \
#                             model.metabolites.get_by_id("VAL_c").formula_weight
# biomass.at["pLEU_b","leaf"]=df_content_5_ProteinAA_proc["L-Leucine (ug/gDW)"]["leaves"]/ \
#                             model.metabolites.get_by_id("LEU_c").formula_weight
# biomass.at["pILE_b","leaf"]=df_content_5_ProteinAA_proc["Isoleucine (ug/gDW)"]["leaves"]/ \
#                             model.metabolites.get_by_id("ILE_c").formula_weight
# biomass.at["pPRO_b","leaf"]=df_content_5_ProteinAA_proc["L-Proline (ug/gDW)"]["leaves"]/ \
#                             model.metabolites.get_by_id("PRO_c").formula_weight
# biomass.at["pMET_b","leaf"]=df_content_5_ProteinAA_proc["L-Methionine (ug/gDW)"]["leaves"]/ \
#                             model.metabolites.get_by_id("MET_c").formula_weight
# biomass.at["pSER_b","leaf"]=df_content_5_ProteinAA_proc["L-Serine (ug/gDW)"]["leaves"]/ \
#                             model.metabolites.get_by_id("SER_c").formula_weight
# biomass.at["pTHR_b","leaf"]=df_content_5_ProteinAA_proc["L-Threonine (ug/gDW)"]["leaves"]/ \
#                             model.metabolites.get_by_id("THR_c").formula_weight
# biomass.at["pPHE_b","leaf"]=df_content_5_ProteinAA_proc["L-Phenylalanine (ug/gDW)"]["leaves"]/ \
#                             model.metabolites.get_by_id("PHE_c").formula_weight
# biomass.at["pASP_b","leaf"]=df_content_5_ProteinAA_proc["L-Aspartic acid (ug/gDW)"]["leaves"]/ \
#                             model.metabolites.get_by_id("L_ASPARTATE_c").formula_weight
# biomass.at["pCYS_b","leaf"]=df_content_5_ProteinAA_proc["L-Cysteine (ug/gDW)"]["leaves"]/ \
#                             model.metabolites.get_by_id("CYS_c").formula_weight
# biomass.at["pGLU_b","leaf"]=df_content_5_ProteinAA_proc["L-Glutamic acid (ug/gDW)"]["leaves"]/ \
#                             model.metabolites.get_by_id("GLT_c").formula_weight
# biomass.at["pASN_b","leaf"]=df_content_5_ProteinAA_proc["L-Asparagine (ug/gDW)"]["leaves"]/ \
#                             model.metabolites.get_by_id("ASN_c").formula_weight
# biomass.at["pLYS_b","leaf"]=df_content_5_ProteinAA_proc["L-Lysine (ug/gDW)"]["leaves"]/ \
#                             model.metabolites.get_by_id("LYS_c").formula_weight
# biomass.at["pGLN_b","leaf"]=df_content_5_ProteinAA_proc["L-Glutamine (ug/gDW)"]["leaves"]/ \
#                             model.metabolites.get_by_id("GLN_c").formula_weight
# biomass.at["pTYR_b","leaf"]=df_content_5_ProteinAA_proc["L-Tyrosine (ug/gDW)"]["leaves"]/ \
#                             model.metabolites.get_by_id("TYR_c").formula_weight
# biomass.at["pTRP_b","leaf"]=df_content_5_ProteinAA_proc["L-Tryptophan (ug/gDW)"]["leaves"]/ \
#                             model.metabolites.get_by_id("TRP_c").formula_weight

In [22]:
biomass

,type,leaf,stem,root,seed
,,,,,
sSUCROSE_b,,0.0,0.0,0.0,0.0
GLC_c,,0.0,0.0,0.0,0.0
FRU_c,,0.0,0.0,0.0,0.0
Starch_b,,0.0,0.0,0.0,0.0
Cellulose_b,,0.0,0.0,0.0,0.0
...,...,...,...,...,...
pALA_b,protein,0.0,0.0,0.0,0.0
pLEU_b,protein,0.0,0.0,0.0,0.0
pPRO_b,protein,0.0,0.0,0.0,0.0


### Stem



Extract diurnal starch, sucrose, hemicellulose and simple sugar content from Clements 1930, Figure 6, 9, 13 and 14. Based on Figure 1 and 2, dawn = 8am and dusk = 8pm

<img src="references/Screenshot_20190717_161155.png" height=50% width=50%>

<img src="references/Screenshot_20190717_160304.png" height=50% width=50%>

<img src="references/Screenshot_20190717_161259.png" height=50% width=50%>

<img src="references/Screenshot_20190717_160242.png" height=50% width=50%>

In [23]:
df_stem_content = pd.DataFrame(data={"Series":[1,2,3,4],
                                     "simple sugars_dawn (g/100gFW)":[0.08928305478569998,1.0069230769230781,
                                                                      0.22941176470588265,0.755800478743633],
                                     "sucrose_dawn (g/100gFW)":[0.24999412110523256,0.10918376068376556,
                                                                0.14117647058823524,0.36680114242250283],
                                     "simple sugars_dusk (g/100gFW)":[0.27423868312757205,0.9930341880341897,
                                                                      0.23368616214394122,0.8292719666520014],
                                     "sucrose_dusk (g/100gFW)":[0,0.08606410256411046,
                                                                0.1748626327321765,0.2553567833200341]})
df_stem_content.set_index("Series",drop=True,inplace=True)
df_stem_content["avg. Δsimple sugars (g/100gFW)"] = df_stem_content["simple sugars_dusk (g/100gFW)"]- \
                                                        df_stem_content["simple sugars_dawn (g/100gFW)"]
df_stem_content["avg. Δsucrose (g/100gFW)"] = df_stem_content["sucrose_dusk (g/100gFW)"]- \
                                                        df_stem_content["sucrose_dawn (g/100gFW)"]

xlist = list()
ylist = list()
ylist2 = list()
for i in df_matureleaf_content_1.index:
    xlist.append(i)
    ylist.append(min(df_stem_content["simple sugars_dawn (g/100gFW)"][i],
                     df_stem_content["simple sugars_dusk (g/100gFW)"][i]))
    ylist2.append(min(df_stem_content["sucrose_dawn (g/100gFW)"][i],
                      df_stem_content["sucrose_dusk (g/100gFW)"][i]))
df_stem_content_std = pd.DataFrame(data={"Series":xlist,
                                       "baseline sucrose (g/100gFW)":ylist2,
                                       "baseline simple sugars (g/100gFW)":ylist})
df_stem_content_std.set_index("Series",drop=True,inplace=True)
df_stem_content_std

,baseline sucrose (g/100gFW),baseline simple sugars (g/100gFW)
Series,,
1,0.000000,0.089283
2,0.086064,0.993034
3,0.141176,0.229412
4,0.255357,0.755800


### Summary

- Sucrose, hexose content data is per FW and conversion to DW is not available

Assumptions

In [24]:
# biomass.at["sSUCROSE_b","stem"] = 1000*find_average(df_stem_content_std["baseline sucrose (g/100gFW)"])/ \
#                                         model.metabolites.get_by_id("SUCROSE_c").formula_weight
# biomass.at["GLC_c","stem"]= 1000*find_average(df_stem_content_std["baseline simple sugars (g/100gFW)"])/ \
#                                         model.metabolites.get_by_id("GLC_c").formula_weight


### Root

Gather root exudate content from Canarini et al 2016, Figure 2

<img src="references/Screenshot_20190718_145459.png" height=50% width=50%>

In [25]:
print("amount of C exudate ="+str(8.689956331877875)+" ug/gDWroot/2h")
print("amount of N exudate ="+str(1.315789473684226)+" ug/gDWroot/2h")

df_root_exudate_1 = pd.DataFrame(data={"%C or %N":["%C","%N"],"Sugars":[14.947368421052634,0],
                                       "Amino acids":[25.684210526315788 - 14.947368421052634,
                                                      67.89473684210535 - 25.789473684210833],
                                       "Organic acids":[30.105263157894733 - 25.684210526315788,0],
                                       "Inorganic ions":[0,67.89473684210535 - 25.789473684210833]})
df_root_exudate_1.set_index("%C or %N",inplace=True,drop=True)
df_root_exudate_1

amount of C exudate =8.689956331877875 ug/gDWroot/2h
amount of N exudate =1.315789473684226 ug/gDWroot/2h


,Sugars,Amino acids,Organic acids,Inorganic ions
%C or %N,,,,
%C,14.947368,10.736842,4.421053,0.000000
%N,0.000000,42.105263,0.000000,42.105263


Gathering fatty-acid content from Thomas et al 2007 Table I

In [26]:
df_root_content_1 = pd.DataFrame(data={"Tissue type":["Conrad - Epidermis","OX760-6 - Epidermis",
                                                      "Conrad - Endodermis","OX760-6 - Endodermis"],
                                       "Fatty acid 16:1":[0.21,0.05,0.82,0.39],
                                       "Fatty acid 16:0":[0.39,0.25,1.50,0.72],
                                       "Fatty acid 18:1":[0.71,0.18,7.46,1.46],
                                       "Fatty acid 18:0":[0.32,0.04,0.83,0.30],
                                       "Fatty acid 20:0":[0.05,0.03,0.28,0.49],
                                       "Fatty acid 22:0":[0.09,0.04,0.31,0.31],
                                       "Total (ug/cm2)":[1.77,0.59,11.20,3.67]})
df_root_content_1.set_index("Tissue type",inplace=True,drop=True)
df_root_content_1

,Fatty acid 16:1,Fatty acid 16:0,Fatty acid 18:1,Fatty acid 18:0,Fatty acid 20:0,Fatty acid 22:0,Total (ug/cm2)
Tissue type,,,,,,,
Conrad - Epidermis,0.21,0.39,0.71,0.32,0.05,0.09,1.77
OX760-6 - Epidermis,0.05,0.25,0.18,0.04,0.03,0.04,0.59
Conrad - Endodermis,0.82,1.50,7.46,0.83,0.28,0.31,11.20
OX760-6 - Endodermis,0.39,0.72,1.46,0.30,0.49,0.31,3.67


In [27]:
# biomass.at["PALMITATE_c","root"] = find_average(df_root_content_1["Fatty acid 16:0"])
# biomass.at["CPD_9245_p","root"] = find_average(df_root_content_1["Fatty acid 16:1"])
# biomass.at["STEARIC_ACID_p","root"] = find_average(df_root_content_1["Fatty acid 18:0"])
# biomass.at["OLEATE_CPD_p","root"] = find_average(df_root_content_1["Fatty acid 18:1"])
# biomass.at["ARACHIDIC_ACID_p","root"] = find_average(df_root_content_1["Fatty acid 20:0"])
# biomass.at["DOCOSANOATE_p","root"] = find_average(df_root_content_1["Fatty acid 22:0"])

### Summary

- sugars, amino acids and organic acids as %C and %N data could not be used directly.
- lipid composition in ug/cm2 cannot be translated to mg/gDW

### Seed


Gather non-reducing sugars, reducing sugars, starch, proteins, phospholipids, glycolipids, sterols and free fatty acid content from Guleria et al 2008, Table 1

In [28]:
df_seed_content = pd.DataFrame(data={"seed type":["large","small"],
                               "starch (mg/gDW)":[49.7,42.3],
                               "reducing sugars (mg/gDW)":[24.9,27.7],
                               "total soluble sugars (mg/gDW)":[125,101],
                               "proteins (mg/gDW)":[377,264],
                               "lipid content (mg/gDW)":[241,224],
                               "phospholipids (mg/gDW lipid)":[57.1,58.7],
                               "glycolipids (mg/gDW lipid)":[10.6,10.3],
                               "sterols (mg/gDW lipid)":[26.8,26.4],
                               "free fatty acids (mg/gDW lipid)":[0.1,0.1],
                               "16:0 (mg/gDW fatty acds)":[132,122],
                               "18:0 (mg/gDW fatty acds)":[29,38],
                               "18:1 (mg/gDW fatty acds)":[269,245],
                               "18:2 (mg/gDW fatty acds)":[513,518],
                               "18:3 (mg/gDW fatty acds)":[56,76]})
df_seed_content_std = pd.DataFrame(data={"seed type":["large","small"],
                               "starch (mg/gDW)":[49.7,42.3],
                               "reducing sugars (mg/gDW)":[24.9,27.7],
                               "non-reducing sugars (mg/gDW)":df_seed_content["total soluble sugars (mg/gDW)"]-\
                                         df_seed_content["reducing sugars (mg/gDW)"],
                               "proteins (mg/gDW)":[377,264],
                               "phospholipids (mg/gDW)":df_seed_content["phospholipids (mg/gDW lipid)"]* \
                                   df_seed_content["lipid content (mg/gDW)"],
                               "glycolipids (mg/gDW)":df_seed_content["glycolipids (mg/gDW lipid)"]* \
                                   df_seed_content["lipid content (mg/gDW)"],
                               "sterols (mg/gDW)":df_seed_content["sterols (mg/gDW lipid)"]* \
                                   df_seed_content["lipid content (mg/gDW)"],
                               "16:0 (mg/gDW)":df_seed_content["16:0 (mg/gDW fatty acds)"]* \
                                         df_seed_content["free fatty acids (mg/gDW lipid)"]* \
                                         df_seed_content["lipid content (mg/gDW)"],
                               "18:0 (mg/gDW)":df_seed_content["18:0 (mg/gDW fatty acds)"]* \
                                         df_seed_content["free fatty acids (mg/gDW lipid)"]* \
                                         df_seed_content["lipid content (mg/gDW)"],
                               "18:1 (mg/gDW)":df_seed_content["18:1 (mg/gDW fatty acds)"]* \
                                         df_seed_content["free fatty acids (mg/gDW lipid)"]* \
                                         df_seed_content["lipid content (mg/gDW)"],
                               "18:2 (mg/gDW)":df_seed_content["18:2 (mg/gDW fatty acds)"]* \
                                         df_seed_content["free fatty acids (mg/gDW lipid)"]* \
                                         df_seed_content["lipid content (mg/gDW)"],
                               "18:3 (mg/gDW)":df_seed_content["18:3 (mg/gDW fatty acds)"]* \
                                         df_seed_content["free fatty acids (mg/gDW lipid)"]* \
                                         df_seed_content["lipid content (mg/gDW)"]})
df_seed_content_std.set_index("seed type",inplace=True,drop=True)
df_seed_content_std

,starch (mg/gDW),reducing sugars (mg/gDW),non-reducing sugars (mg/gDW),proteins (mg/gDW),phospholipids (mg/gDW),glycolipids (mg/gDW),sterols (mg/gDW),16:0 (mg/gDW),18:0 (mg/gDW),18:1 (mg/gDW),18:2 (mg/gDW),18:3 (mg/gDW)
seed type,,,,,,,,,,,,
large,49.7,24.9,100.1,377,13761.1,2554.6,6458.8,3181.2,698.9,6482.9,12363.3,1349.6
small,42.3,27.7,73.3,264,13148.8,2307.2,5913.6,2732.8,851.2,5488.0,11603.2,1702.4


Extract FW/DW, lipid, protein, starch, soluble sugar and reducing sugar content from Sharma et al 2013, Figure 1 and 2


<img src="references/Screenshot_20190719_101524.png" height=25% width=25%>

<img src="references/Screenshot_20190719_101729.png" height=25% width=25%>

In [29]:
df_seed_content_2 = pd.DataFrame(data={"node position":range(4,18),
                               "DW/FW":[0.928872804935928,0.927787138111058,0.934610821072615,
                                        0.938872804935928,0.937322021831989,0.918329378262933,
                                        0.95515068818225,0.944299952539155,0.945538680588514,
                                        0.948406502135738,0.948249881347888,0.948559563360228,
                                        0.956778595158994,0.958483626008543],
                               "lipid content (mg/gDW)":[242.2236872277,264.320110066498,255.514790185737,
                                                         247.835932125659,237.345218986471,237.531529465719,
                                                         235.470648933731,225.550332492547,212.250630589314,
                                                         211.313345562944,207.564205457464,207.750515936712,
                                                         216.922724145838,221.603416647558],
                               "proteins (mg/gDW)":[251.291824869483,241.120372567632,245.770052206929,
                                                    222.503856193641,236.746559088752,239.656502135738,
                                                    269.587980541054,271.621381110584,278.882890365449,
                                                    334.986058376839,327.426435690555,323.350735643094,
                                                    316.663205980066,321.312885619364],
                               "starch (mg/gDW)":[42.5131154156578,40.2199354317999,48.283393866021,
                                                  37.240213882163,41.1970338983051,45.3324253430186,
                                                  42.681598062954,40.3894269572236,39.7033898305085,
                                                  49.195419693301,40.4736682808717,42.8233454398709,
                                                  39.8153753026634,35.5579096045198],
                               "total soluble sugars (mg/gDW)":[90.6887971663593,88.8516144140979,
                                                                88.4657943268639,105.697140013466,
                                                                81.2716255378942,91.6635929861538,
                                                                98.7394982582477,91.7227247445918,
                                                                86.7779046280847,86.8071777758262,
                                                                97.1985597611311,90.5968794824508,
                                                                81.5069816457364,90.6548403149791],
                               "reducing sugars (mg/gDW)":[112.999279159929,151.966724463752,
                                                           122.654639677375,137.206647314383,
                                                           155.484034366537,157.207036957665,
                                                           115.897056245007,65.4822809717704,
                                                           49.4141713261512,53.6188119776346,
                                                           58.6545617487192,78.173352295973,
                                                           46.7922616844281,52.2406436907015]})
df_seed_content_2.set_index("node position",drop=True,inplace=True)
df_seed_content_2

,DW/FW,lipid content (mg/gDW),proteins (mg/gDW),starch (mg/gDW),total soluble sugars (mg/gDW),reducing sugars (mg/gDW)
node position,,,,,,
4,0.928873,242.223687,251.291825,42.513115,90.688797,112.999279
5,0.927787,264.320110,241.120373,40.219935,88.851614,151.966724
6,0.934611,255.514790,245.770052,48.283394,88.465794,122.654640
7,0.938873,247.835932,222.503856,37.240214,105.697140,137.206647
8,0.937322,237.345219,236.746559,41.197034,81.271626,155.484034
9,0.918329,237.531529,239.656502,45.332425,91.663593,157.207037
10,0.955151,235.470649,269.587981,42.681598,98.739498,115.897056
11,0.944300,225.550332,271.621381,40.389427,91.722725,65.482281
12,0.945539,212.250631,278.882890,39.703390,86.777905,49.414171


### Summary

Assumptions

- Sharma et al 2013 data has reducing  sugar content higher than total soluble sugar content and so given that the same data was available from Guleria et al 2008, this data was ignored

## Set FA composition first and create a dictionary of models

In [30]:

modelDict = dict()


In [31]:

biomass.at["PALMITATE_p","seed"]=find_average(df_seed_content_std["16:0 (mg/gDW)"])/ \
                                        model.metabolites.get_by_id("PALMITATE_p").formula_weight
biomass.at["STEARIC_ACID_p","seed"]=find_average(df_seed_content_std["18:0 (mg/gDW)"])/ \
                                        model.metabolites.get_by_id("STEARIC_ACID_p").formula_weight
biomass.at["OLEATE_CPD_p","seed"]=find_average(df_seed_content_std["18:1 (mg/gDW)"])/ \
                                        model.metabolites.get_by_id("OLEATE_CPD_p").formula_weight
biomass.at["Octadecadienoate_p","seed"]=find_average(df_seed_content_std["18:2 (mg/gDW)"])/ \
                                        model.metabolites.get_by_id("Octadecadienoate_p").formula_weight
biomass.at["LINOLENIC_ACID_p","seed"]=find_average(df_seed_content_std["18:3 (mg/gDW)"])/ \
                                        model.metabolites.get_by_id("LINOLENIC_ACID_p").formula_weight

In [32]:
def updateFAcomposition(model,organ,biomass):
    
    temp = model.copy()
    from cobra.core import Reaction
    FACP = {"PALMITATE_p":"Palmitoyl_ACPs_p",
            "CPD_9245_p":"Palmitoleoyl_ACP_p",
            "CPD_17412_p":"hexadecadienoate_ACP_p",
            "CPD_17291_p":"hexadecatrienoate_ACP_p",
            "STEARIC_ACID_p":"Stearoyl_ACPs_p",
            "OLEATE_CPD_p":"Oleoyl_ACPs_p",
            "Octadecadienoate_p":"Octadecadienoyl_ACP_p",
            "LINOLENIC_ACID_p":"Octadecatrienoyl_ACP_p",
            "ARACHIDIC_ACID_p":"Arachidoyl_ACPs_p",
            "CPD_16709_p":"Eicosenoyl_ACP_p",
            "DOCOSANOATE_p":"Behenoyl_ACPs_p"}
    
    PLs = ["ACYL_SN_GLYCEROL_3P_p","L_PHOSPHATIDATE_p","L_PHOSPHATIDATE_m","DIACYLGLYCEROL_p",
           "DIACYLGLYCEROL_r","Triacylglycerols_p","PHOSPHATIDYL_CHOLINE_r",
           "L_1_PHOSPHATIDYL_ETHANOLAMINE_r","L_1_PHOSPHATIDYL_GLYCEROL_p",
           "L_1_PHOSPHATIDYL_GLYCEROL_P_p","L_1_PHOSPHATIDYL_GLYCEROL_P_m",
           "L_1_PHOSPHATIDYL_GLYCEROL_m","2_Lysophosphatidylcholines_r",
           "Lysophosphatidylglycerols_r","CDPDIACYLGLYCEROL_p","CDPDIACYLGLYCEROL_m",
           "D_Galactosyl_12_diacyl_glycerols_p","Galactosyl_galactosyl_diacyl_glycerols_p"]


    for met in PLs:
        met=temp.metabolites.get_by_id(met)
        met.formula=""
    
    FAdict = dict(biomass[biomass["type"]=="fattyacid"][organ])

    k = organ
    RXN1 = Reaction("Fatty_acid_mix_"+k)
    RXN2 = Reaction("Fatty_acid_ACP_"+k)
    tot = 0
    for met in FAdict.keys():
        RXN1.add_metabolites({temp.metabolites.get_by_id(met):-1*FAdict[met]})
        RXN2.add_metabolites({temp.metabolites.get_by_id(FACP[met]):-1*FAdict[met]})
        tot = tot+FAdict[met]
    print(tot)
    if tot==0:
        RXN1.add_metabolites({temp.metabolites.PALMITATE_p:-1})
        RXN2.add_metabolites({temp.metabolites.Palmitoyl_ACPs_p:-1})
        tot = 1
    RXN1.add_metabolites({temp.metabolites.Fatty_Acids_p:tot})
    RXN1.lower_bound = 1000
    RXN1.upper_bound = 0
    temp.add_reaction(RXN1)

    RXN2.add_metabolites({temp.metabolites.Fatty_acyl_ACP_p:tot})
    RXN2.lower_bound = 1000
    RXN2.upper_bound = 0
    temp.add_reaction(RXN2)
    
    generateMissingFormula(temp)
    
    return temp

def generateMissingFormula(model,debug=False):
    loop_counter = 0
    former = 0
    for met in model.metabolites:
        if met.formula == "" or met.formula == "NA":
            former = former +1
    latter = 1
    while True:
        loop_counter = loop_counter+1
        if debug:
            print("Loop = "+str(loop_counter))
        former = latter
        for rxn in model.reactions:
            count = 0
            for met in rxn.metabolites:
                if met.formula=="" or met.formula=="NA" or met.formula == None:
                    if met.formula == "NA" or met.formula == None:
                        met.formula = ""
                    count = count + 1
                    coeff = rxn.metabolites[met]
            if count == 1:
                unb = rxn.check_mass_balance()
                eqn = rxn.reaction
                eqn = " "+eqn+" "
                for met in rxn.metabolites.keys():
                    formula = met.formula
                    if formula == None:
                        formula = "0"
                        NF_list.add(rxn.id)
                    eqn=eqn.replace(" "+met.id+" ","("+formula+")")
                if debug:
                    print(eqn)
                    print(unb)
                for met in rxn.metabolites:
                    if met.formula == "":
                        tempForm = ""
                        for a in sorted(unb.keys()):
                            if a=="charge" or round(unb[a],2)==0:
                                continue
                            num = float(abs(unb[a]))/abs(coeff)
                            if str(round(num))==str(num):
                                tempForm = tempForm+a+str(int(round(num)))
                            else:
                                tempForm = tempForm+a+str(num)
                                #print(a)
                                #print(round(num)==num)
                                #print(round(num))
                                #print(num)
                                #print(type(round(num)))
                                #print(type(num))
                        met.formula = tempForm
                        if debug:
                            print(met.id)
                            print(tempForm)
        latter = 0
        for met in model.metabolites:
            if met.formula == "" or met.formula == "NA":
                latter = latter +1
        if former == latter:
            break
        
        

In [33]:
temp = updateFAcomposition(model,"seed",biomass)
modelDict["seed"]=temp

Read LP format model from file C:\Users\sanus\AppData\Local\Temp\tmp7skmybqg.lp
Reading time = 0.02 seconds
: 880 rows, 1846 columns, 9306 nonzeros
83.96167973352584


In [34]:
modelDict["seed"].metabolites.get_by_id("L_PHOSPHATIDATE_p").formula_weight

646.8754409999999

In [35]:
organDict={"leaf":"leaves","stem":"stems","root":"roots"}

for organ in organDict.keys():
    biomass.at["PALMITATE_p",organ]=df_content_5_FA_proc["C16:0 (ug/gDW)"][organDict[organ]]/ \
                                    model.metabolites.get_by_id("PALMITATE_p").formula_weight
    biomass.at["CPD_9245_p",organ]=df_content_5_FA_proc["C16:1z-9 (ug/gDW)"][organDict[organ]]/ \
                                   model.metabolites.get_by_id("CPD_9245_p").formula_weight
    biomass.at["STEARIC_ACID_p",organ]=df_content_5_FA_proc["C18:0 (ug/gDW)"][organDict[organ]]/ \
                                   model.metabolites.get_by_id("STEARIC_ACID_p").formula_weight
    biomass.at["OLEATE_CPD_p",organ]=(df_content_5_FA_proc["C18:1z-9 (ug/gDW)"][organDict[organ]]+ \
                                      df_content_5_FA_proc["C18:1e-9 (ug/gDW)"][organDict[organ]])/ \
                                      model.metabolites.get_by_id("OLEATE_CPD_p").formula_weight
    biomass.at["Octadecadienoate_p",organ]=df_content_5_FA_proc["C18:2z-9z-12 (ug/gDW)"][organDict[organ]]/ \
                                           model.metabolites.get_by_id("Octadecadienoate_p").formula_weight
    biomass.at["ARACHIDIC_ACID_p",organ]=df_content_5_FA_proc["C20:0 (ug/gDW)"][organDict[organ]]/ \
                                           model.metabolites.get_by_id("ARACHIDIC_ACID_p").formula_weight
    biomass.at["CPD_16709_p",organ]=df_content_5_FA_proc["C20:1z-13 (ug/gDW)"][organDict[organ]]/ \
                                           model.metabolites.get_by_id("CPD_16709_p").formula_weight
    biomass.at["DOCOSANOATE_p",organ]=df_content_5_FA_proc["C22:0 (ug/gDW)"][organDict[organ]]/ \
                                           model.metabolites.get_by_id("DOCOSANOATE_p").formula_weight
    
    temp = updateFAcomposition(model,organ,biomass)
    modelDict[organ]=temp

Read LP format model from file C:\Users\sanus\AppData\Local\Temp\tmpzhq29rwd.lp
Reading time = 0.03 seconds
: 880 rows, 1846 columns, 9306 nonzeros
32.38817197039607
Read LP format model from file C:\Users\sanus\AppData\Local\Temp\tmpk8w_j5tf.lp
Reading time = 0.02 seconds
: 880 rows, 1846 columns, 9306 nonzeros
8.185775383471956
Read LP format model from file C:\Users\sanus\AppData\Local\Temp\tmp4fhou_sr.lp
Reading time = 0.02 seconds
: 880 rows, 1846 columns, 9306 nonzeros
7.507823912632633


In [36]:
biomass.at["sSUCROSE_b","seed"] = find_average(df_seed_content_std["non-reducing sugars (mg/gDW)"])/ \
                                        modelDict["seed"].metabolites.get_by_id("SUCROSE_c").formula_weight
biomass.at["GLC_c","seed"]=find_average(df_seed_content_std["reducing sugars (mg/gDW)"])/ \
                                        modelDict["seed"].metabolites.get_by_id("GLC_c").formula_weight
biomass.at["L_PHOSPHATIDATE_p","seed"]=find_average(df_seed_content_std["phospholipids (mg/gDW)"])/ \
                                        modelDict["seed"].metabolites.get_by_id("L_PHOSPHATIDATE_p").formula_weight
biomass.at["Starch_b","seed"]=find_average(df_seed_content_std["starch (mg/gDW)"])/ \
                                        modelDict["seed"].metabolites.get_by_id("GLC_c").formula_weight
rxn = modelDict["seed"].reactions.Protein_biomass_tx
proteinMol = find_average(df_seed_content_std["proteins (mg/gDW)"])/proteinMW
biomass.at["pHIS_b","seed"]=abs(rxn.metabolites[modelDict["seed"].metabolites.get_by_id("pHIS_b")])*proteinMol
biomass.at["pILE_b","seed"]=abs(rxn.metabolites[modelDict["seed"].metabolites.get_by_id("pILE_b")])*proteinMol
biomass.at["pTHR_b","seed"]=abs(rxn.metabolites[modelDict["seed"].metabolites.get_by_id("pTHR_b")])*proteinMol
biomass.at["pARG_b","seed"]=abs(rxn.metabolites[modelDict["seed"].metabolites.get_by_id("pARG_b")])*proteinMol
biomass.at["pASN_b","seed"]=abs(rxn.metabolites[modelDict["seed"].metabolites.get_by_id("pASN_b")])*proteinMol
biomass.at["pGLU_b","seed"]=abs(rxn.metabolites[modelDict["seed"].metabolites.get_by_id("pGLU_b")])*proteinMol
biomass.at["pPHE_b","seed"]=abs(rxn.metabolites[modelDict["seed"].metabolites.get_by_id("pPHE_b")])*proteinMol
biomass.at["pGLN_b","seed"]=abs(rxn.metabolites[modelDict["seed"].metabolites.get_by_id("pGLN_b")])*proteinMol
biomass.at["pTYR_b","seed"]=abs(rxn.metabolites[modelDict["seed"].metabolites.get_by_id("pTYR_b")])*proteinMol
biomass.at["pMET_b","seed"]=abs(rxn.metabolites[modelDict["seed"].metabolites.get_by_id("pMET_b")])*proteinMol
biomass.at["pASP_b","seed"]=abs(rxn.metabolites[modelDict["seed"].metabolites.get_by_id("pASP_b")])*proteinMol
biomass.at["pVAL_b","seed"]=abs(rxn.metabolites[modelDict["seed"].metabolites.get_by_id("pVAL_b")])*proteinMol
biomass.at["pLYS_b","seed"]=abs(rxn.metabolites[modelDict["seed"].metabolites.get_by_id("pLYS_b")])*proteinMol
biomass.at["pSER_b","seed"]=abs(rxn.metabolites[modelDict["seed"].metabolites.get_by_id("pSER_b")])*proteinMol
biomass.at["pGLY_b","seed"]=abs(rxn.metabolites[modelDict["seed"].metabolites.get_by_id("pGLY_b")])*proteinMol
biomass.at["pALA_b","seed"]=abs(rxn.metabolites[modelDict["seed"].metabolites.get_by_id("pALA_b")])*proteinMol
biomass.at["pLEU_b","seed"]=abs(rxn.metabolites[modelDict["seed"].metabolites.get_by_id("pLEU_b")])*proteinMol


In [37]:
organDict={"leaf":"leaves","stem":"stems","root":"roots"}

for organ in organDict.keys():
    biomass.at["sSUCROSE_b",organ]=df_content_5_soluble_proc["Sucrose (ug/gDW)"][organDict[organ]]/ \
                                    modelDict[organ].metabolites.get_by_id("SUCROSE_c").formula_weight
    biomass.at["GLC_c",organ]=df_content_5_soluble_proc["Glucose (ug/gDW)"][organDict[organ]]/ \
                               modelDict[organ].metabolites.get_by_id("GLC_c").formula_weight
    biomass.at["FRU_c",organ]=df_content_5_soluble_proc["Fructose (ug/gDW)"][organDict[organ]]/ \
                               modelDict[organ].metabolites.get_by_id("FRU_c").formula_weight

    biomass.at["Starch_b",organ]=df_content_5_starch_proc["Glucose (ug/gDW)"][organDict[organ]]/ \
                                  modelDict[organ].metabolites.get_by_id("GLC_c").formula_weight

    biomass.at["Cellulose_b",organ]=df_content_5_cellwall_proc["Mannose (ug/gDW)"][organDict[organ]]/ \
                                     modelDict[organ].metabolites.get_by_id("MANNOSE_c").formula_weight
    biomass.at["Xylan_b",organ]=df_content_5_cellwall_proc["Xylose (ug/gDW)"][organDict[organ]]/ \
                                 modelDict[organ].metabolites.get_by_id("Xylan_b").formula_weight
    
    
    biomass.at["L_PHOSPHATIDATE_p",organ]=df_content_5_PL_proc["Lipid (ug/gDW)"][organDict[organ]]/ \
                                           modelDict[organ].metabolites.get_by_id("L_PHOSPHATIDATE_p").formula_weight

    biomass.at["sMAL_b",organ]=df_content_5_soluble_proc["Malate (ug/gDW)"][organDict[organ]]/ \
                                modelDict[organ].metabolites.get_by_id("MAL_c").formula_weight
    biomass.at["sCIT_b",organ]=df_content_5_soluble_proc["Citrate (ug/gDW)"][organDict[organ]]/ \
                                modelDict[organ].metabolites.get_by_id("CIT_c").formula_weight
    biomass.at["sFUM_b",organ]=df_content_5_soluble_proc["Fumarate (ug/gDW)"][organDict[organ]]/ \
                                modelDict[organ].metabolites.get_by_id("FUM_c").formula_weight

    biomass.at["VAL_c",organ]=df_content_5_soluble_proc["Valine (ug/gDW)"][organDict[organ]]/ \
                               modelDict[organ].metabolites.get_by_id("VAL_c").formula_weight
    biomass.at["sALA_b",organ]=df_content_5_soluble_proc["Alanine (ug/gDW)"][organDict[organ]]/ \
                                modelDict[organ].metabolites.get_by_id("sALA_b").formula_weight
    biomass.at["LEU_c",organ]=df_content_5_soluble_proc["Leucine (ug/gDW)"][organDict[organ]]/ \
                               modelDict[organ].metabolites.get_by_id("LEU_c").formula_weight
    biomass.at["ILE_c",organ]=df_content_5_soluble_proc["Isoleucine (ug/gDW)"][organDict[organ]]/ \
                               modelDict[organ].metabolites.get_by_id("ILE_c").formula_weight
    biomass.at["PRO_c",organ]=df_content_5_soluble_proc["Proline (ug/gDW)"][organDict[organ]]/ \
                               modelDict[organ].metabolites.get_by_id("PRO_c").formula_weight
    biomass.at["GLY_c",organ]=df_content_5_soluble_proc["Glycine (ug/gDW)"][organDict[organ]]/ \
                               modelDict[organ].metabolites.get_by_id("GLY_c").formula_weight
    biomass.at["SUC_c",organ]=df_content_5_soluble_proc["Succinate (ug/gDW)"][organDict[organ]]/ \
                                modelDict[organ].metabolites.get_by_id("SUC_c").formula_weight
    biomass.at["sSER_b",organ]=df_content_5_soluble_proc["Serine (ug/gDW)"][organDict[organ]]/ \
                                modelDict[organ].metabolites.get_by_id("sSER_b").formula_weight
    biomass.at["THR_c",organ]=df_content_5_soluble_proc["Threonine (ug/gDW)"][organDict[organ]]/ \
                               modelDict[organ].metabolites.get_by_id("THR_c").formula_weight
    biomass.at["MET_c",organ]=df_content_5_soluble_proc["Methionine (ug/gDW)"][organDict[organ]]/ \
                               modelDict[organ].metabolites.get_by_id("MET_c").formula_weight
    biomass.at["sASP_b",organ]=df_content_5_soluble_proc["Aspartate (ug/gDW)"][organDict[organ]]/ \
                                modelDict[organ].metabolites.get_by_id("sASP_b").formula_weight
    biomass.at["sGABA_b",organ]=df_content_5_soluble_proc["GABA (ug/gDW)"][organDict[organ]]/ \
                                 modelDict[organ].metabolites.get_by_id("sGABA_b").formula_weight
    biomass.at["CYS_c",organ]=df_content_5_soluble_proc["Cysteine (ug/gDW)"][organDict[organ]]/ \
                               modelDict[organ].metabolites.get_by_id("CYS_c").formula_weight
    biomass.at["PHE_c",organ]=df_content_5_soluble_proc["Phenylalanine (ug/gDW)"][organDict[organ]]/ \
                               modelDict[organ].metabolites.get_by_id("PHE_c").formula_weight
    biomass.at["sGLU_b",organ]=df_content_5_soluble_proc["Glutamate (ug/gDW)"][organDict[organ]]/ \
                                modelDict[organ].metabolites.get_by_id("sGLU_b").formula_weight
    biomass.at["sCIT_b",organ]=df_content_5_soluble_proc["Citrate (ug/gDW)"][organDict[organ]]/ \
                                modelDict[organ].metabolites.get_by_id("sCIT_b").formula_weight
    biomass.at["LYS_c",organ]=df_content_5_soluble_proc["Lysine (ug/gDW)"][organDict[organ]]/ \
                               modelDict[organ].metabolites.get_by_id("LYS_c").formula_weight
    biomass.at["TYR_c",organ]=df_content_5_soluble_proc["Tyrosine (ug/gDW)"][organDict[organ]]/ \
                               modelDict[organ].metabolites.get_by_id("TYR_c").formula_weight
    biomass.at["MYO_INOSITOL_c",organ]=df_content_5_soluble_proc["myoInositol (ug/gDW)"][organDict[organ]]/ \
                                        modelDict[organ].metabolites.get_by_id("MYO_INOSITOL_c").formula_weight


    biomass.at["pALA_b",organ]=df_content_5_ProteinAA_proc["L-Alanine (ug/gDW)"][organDict[organ]]/ \
                                modelDict[organ].metabolites.get_by_id("L_ALPHA_ALANINE_c").formula_weight
    biomass.at["pGLY_b",organ]=df_content_5_ProteinAA_proc["Glycine (ug/gDW)"][organDict[organ]]/ \
                                modelDict[organ].metabolites.get_by_id("GLY_c").formula_weight
    biomass.at["pVAL_b",organ]=df_content_5_ProteinAA_proc["L-Valine (ug/gDW)"][organDict[organ]]/ \
                                modelDict[organ].metabolites.get_by_id("VAL_c").formula_weight
    biomass.at["pLEU_b",organ]=df_content_5_ProteinAA_proc["L-Leucine (ug/gDW)"][organDict[organ]]/ \
                                modelDict[organ].metabolites.get_by_id("LEU_c").formula_weight
    biomass.at["pILE_b",organ]=df_content_5_ProteinAA_proc["Isoleucine (ug/gDW)"][organDict[organ]]/ \
                                modelDict[organ].metabolites.get_by_id("ILE_c").formula_weight
    biomass.at["pPRO_b",organ]=df_content_5_ProteinAA_proc["L-Proline (ug/gDW)"][organDict[organ]]/ \
                                modelDict[organ].metabolites.get_by_id("PRO_c").formula_weight
    biomass.at["pMET_b",organ]=df_content_5_ProteinAA_proc["L-Methionine (ug/gDW)"][organDict[organ]]/ \
                                modelDict[organ].metabolites.get_by_id("MET_c").formula_weight
    biomass.at["pSER_b",organ]=df_content_5_ProteinAA_proc["L-Serine (ug/gDW)"][organDict[organ]]/ \
                                modelDict[organ].metabolites.get_by_id("SER_c").formula_weight
    biomass.at["pTHR_b",organ]=df_content_5_ProteinAA_proc["L-Threonine (ug/gDW)"][organDict[organ]]/ \
                                modelDict[organ].metabolites.get_by_id("THR_c").formula_weight
    biomass.at["pPHE_b",organ]=df_content_5_ProteinAA_proc["L-Phenylalanine (ug/gDW)"][organDict[organ]]/ \
                                modelDict[organ].metabolites.get_by_id("PHE_c").formula_weight
    biomass.at["pASP_b",organ]=df_content_5_ProteinAA_proc["L-Aspartic acid (ug/gDW)"][organDict[organ]]/ \
                                modelDict[organ].metabolites.get_by_id("L_ASPARTATE_c").formula_weight
    biomass.at["pCYS_b",organ]=df_content_5_ProteinAA_proc["L-Cysteine (ug/gDW)"][organDict[organ]]/ \
                                modelDict[organ].metabolites.get_by_id("CYS_c").formula_weight
    biomass.at["pGLU_b",organ]=df_content_5_ProteinAA_proc["L-Glutamic acid (ug/gDW)"][organDict[organ]]/ \
                                modelDict[organ].metabolites.get_by_id("GLT_c").formula_weight
    biomass.at["pASN_b",organ]=df_content_5_ProteinAA_proc["L-Asparagine (ug/gDW)"][organDict[organ]]/ \
                                modelDict[organ].metabolites.get_by_id("ASN_c").formula_weight
    biomass.at["pLYS_b",organ]=df_content_5_ProteinAA_proc["L-Lysine (ug/gDW)"][organDict[organ]]/ \
                                modelDict[organ].metabolites.get_by_id("LYS_c").formula_weight
    biomass.at["pGLN_b",organ]=df_content_5_ProteinAA_proc["L-Glutamine (ug/gDW)"][organDict[organ]]/ \
                                modelDict[organ].metabolites.get_by_id("GLN_c").formula_weight
    biomass.at["pTYR_b",organ]=df_content_5_ProteinAA_proc["L-Tyrosine (ug/gDW)"][organDict[organ]]/ \
                                modelDict[organ].metabolites.get_by_id("TYR_c").formula_weight
    biomass.at["pTRP_b",organ]=df_content_5_ProteinAA_proc["L-Tryptophan (ug/gDW)"][organDict[organ]]/ \
                                modelDict[organ].metabolites.get_by_id("TRP_c").formula_weight

In [38]:
biomass.to_csv("Data/biomass_soy.csv")
biomass

,type,leaf,stem,root,seed
,,,,,
sSUCROSE_b,,0.000007,0.000007,0.000006,0.253289
GLC_c,,0.000035,0.000107,0.000051,0.145985
FRU_c,,0.000602,0.000878,0.000955,0.000000
Starch_b,,3.582083,2.740753,1.827811,0.255334
Cellulose_b,,0.504756,0.662413,0.648044,0.000000
...,...,...,...,...,...
pALA_b,protein,3.998875,4.045921,3.039394,0.161649
pLEU_b,protein,3.223817,2.893927,2.262680,0.003711
pPRO_b,protein,2.553908,3.042380,2.293533,0.000000


### References

- Liu, F., Jensen, C.R. and Andersen, M.N. (2004) Drought stress effect on carbohydrate concentration in soybean leaves and pods during early reproductive development: its implication in altering pod set. F. Crop. Res., 86, 1–13. https://doi.org/10.1016/S0378-4290(03)00165-5.
- Chiozza, M. V, O’Neal, M.E. and MacIntosh, G.C. (2010) Constitutive and Induced Differential Accumulation of Amino Acid in Leaves of Susceptible and Resistant Soybean Plants in Response to the Soybean Aphid (Hemiptera: Aphididae). Environ. Entomol., 39, 856–864. https://doi.org/10.1603/EN09338.
- Huber, S.C. (2008) Biochemical Mechanism for Regulation of Sucrose Accumulation in Leaves during Photosynthesis. PLANT Physiol., 91, 656–662. https://doi.org/10.1104/pp.91.2.656.
- White, M.C., Decker, A.M. and Chaney, R.L. (2008) Metal Complexation in Xylem Fluid : I. CHEMICAL COMPOSITION OF TOMATO AND SOYBEAN STEM EXUDATE. PLANT Physiol., 67, 292–300. https://doi.org/10.1104/pp.67.2.292.
- Clements, H.F. (1930) Hourly Variations in Carbohydrate Content of Leaves and Petioles. Bot. Gaz., 89, 241–272. https://www.jstor.org/stable/2471041.
- Canarini, A., Merchant, A. and Dijkstra, F.A. (2016) Drought effects on Helianthus annuus and Glycine max metabolites: from phloem to root exudates. Rhizosphere, 2, 85–97. https://doi.org/10.1016/j.rhisph.2016.06.003.
- Thomas, R., Fang, X., Ranathunge, K., Anderson, T.R., Peterson, C.A. and Bernards, M.A. (2007) Soybean Root Suberin: Anatomical Distribution, Chemical Composition, and Relationship to Partial Resistance to Phytophthora sojae. PLANT Physiol., 144, 299–311. https:doi.org/10.1104/pp.106.091090.
- Guleria, S., Sharma, S., Gill, B.S. and Munshi, S.K. (2008) Distribution and biochemical composition of large and small seeds of soybean (Glycine max L.). J. Sci. Food Agric., 88, 269–272. https://doi.org/10.1002/jsfa.2825.
- Sharma, S., Kaur, A., Bansal, A. and Gill, B.S. (2013) Positional effects on soybean seed composition during storage. J. Food Sci. Technol., 50, 353–359. https://doi.org/10.1007/s13197-011-0341-0.